In [1]:
from database import model as m
from datetime import datetime, timedelta
from sqlalchemy import desc
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
import boto3
import base64
from botocore.exceptions import ClientError
import json

In [2]:
def get_secret():
    secret_name = "SecretCorona"
    region_name = "eu-west-1"
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.
    get_secret_value_response = client.get_secret_value(
        SecretId=secret_name
    )
    return get_secret_value_response

secret = json.loads(get_secret()["SecretString"])

engine = create_engine(
    'mssql+pymssql://' +
    secret['username'] + ':' + secret['password'] + '@' + secret['host'] + ':' +
    str(secret['port']) + '/Corona'

)

session = sessionmaker()(bind=engine)

In [7]:
df =pd.read_csv("countermeasures_db_johnshopkins.csv")

In [8]:
df.drop(['Confirmed Cases', 'Deaths','Unnamed: 0'], axis=1, inplace=True)

df = df.melt(id_vars = ["Date","Country"])

df["Date"] =df["Date"].apply(lambda x: pd.to_datetime(x))

df = df[~pd.isnull(df["value"])]

In [105]:
df.sort_values(by=['Country'],inplace = True)

In [106]:
df= df.reset_index()

In [109]:
for i in range(len(df)):
    
    if i == 0:
        Country = session.query(m.Country).filter(m.Country.country == df["Country"][i]).first()
        
        CounterMeasures = m.CounterMeasures( 
                            date = df["Date"][i],
                            measure = df["variable"][i],
                            value =df["value"][i],
                            country_id = Country.id)


        session.add(CounterMeasures)

        
    else:

    
        if df["Country"][i] != df["Country"][i-1]:
            
            Country = session.query(m.Country).filter(m.Country.country == df["Country"][i]).first()
            
            if Country is None:
                print(df["Country"][i])
                
            else:

                CounterMeasures = m.CounterMeasures( 
                                date = df["Date"][i],
                                measure = df["variable"][i],
                                value =df["value"][i],
                                country_id = Country.id)


                session.add(CounterMeasures)

        else:
            
            if Country is None:
                print(df["Country"][i])
                
            else:


                CounterMeasures = m.CounterMeasures( 
                                    date = df["Date"][i],
                                    measure = df["variable"][i],
                                    value =df["value"][i],
                                    country_id = Country.id)


                session.add(CounterMeasures)

    if i%1000 == 0:
        session.commit()
        print("Commiting")
    else:
        if  i%100 == 0:
            print(i)
        
    
session.commit()
session.close()

Commiting
100
200
300
400
500
600
700
800
900
Commiting
1100
1200
1300
1400
1500
1600
1700
1800
1900
Commiting
2100
2200
2300
2400
2500
2600
2700
2800
2900
Commiting
3100
3200
3300
3400
3500
3600
3700
3800
3900
Commiting
4100
4200
4300
4400
4500
4600
4700
4800
4900
Commiting
5100
5200
5300
5400
5500
5600
5700
5800
5900
Commiting
6100
6200
6300
6400
6500
6600
6700
6800
6900
Commiting
7100
7200
7300
7400
7500
7600
7700
7800
7900
Commiting
8100
8200
8300
8400
8500
8600
8700
8800
8900
Commiting
9100
9200
9300
9400
9500
9600
9700
9800
9900
Commiting
10100
10200
10300
10400
10500
10600
10700
10800
10900
Commiting
11100
11200
11300
11400
11500
11600
11700
11800
11900
Commiting
12100
12200
12300
12400
12500
12600
12700
12800
12900
Commiting
13100
13200
13300
13400
13500
13600
13700
13800
13900
Commiting
14100
14200
14300
14400
14500
14600
14700
14800
14900
Commiting
15100
15200
15300
15400
15500
15600
15700
15800
15900
Commiting
16100
16200
16300
16400
16500
16600
16700
16800
16900
Commiting
1